In [1]:
import numpy as np
import pandas as pd
import glob
import itertools

from helper import *

In [ ]:
log_dirs = glob.glob("test1/m1.7_z0.015_v0_var*_covergence")

M_sample = [1.7]
Z_sample = [0.015]
V_sample = [0]
var_samples = [True, False]


combinations = itertools.product(M_sample, Z_sample, V_sample, var_samples)
data_dict = {}

k = 0
for m, z, v, var in combinations:
    print(m, z, v, var)
    key = f"M={m}_Z={z}_V={v}_var={var}"
    hist, freqs, profs, n_profs, prof_index = get_data(log_dirs[k])
    models = [i[0] for i in prof_index if i[2] in n_profs]
    indexes = hist.index[hist.model_number.isin(models)]
    ages = hist.star_age[indexes]/1e6
    data_dict[key] = pd.DataFrame()
    data_dict[key]['age'] = hist['star_age']/1e6
    data_dict[key]['log_cntr_Rho'] = hist['log_cntr_Rho'].values
    data_dict[key]['log_cntr_T'] = hist['log_cntr_T'].values
    data_dict[key]['log_Teff'] = hist['log_Teff'].values
    data_dict[key] = pd.merge(data_dict[key], get_freqs(freqs, ages, k), on="age")
    ages = np.arange(0, data_dict[key].iloc[-1]["age"], 0.1)
    new_df = pd.DataFrame()
    new_df["age"] = ages
    for column in data_dict[key].columns[1:]:
        new_df[column] = interp(data_dict[key]["age"], data_dict[key][column], ages)
    data_dict[key] = new_df
    k += 1